In [1]:
from pathlib import Path
import czifile
import napari
import numpy as np
import skimage
from skimage import measure, exposure
from skimage.measure import regionprops_table
import pandas as pd
import pyclesperanto_prototype as cle
import matplotlib.pyplot as plt
import gc

# Select a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
# cle.select_device("RTX")

In [2]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    images.append(str(file_path))
    
images

['raw_data\\well 1_3.czi']

In [3]:
image = images[0]

# Extract filename and well_id
file_path = Path(image)
filename = file_path.stem
well_id = filename.split(" ")[1]

# Read the image file and remove singleton dimensions
img = czifile.imread(image)
img = img.squeeze()

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = 2 # Use 2 or 4 for compression (None for lossless)

# Extract the stack containing the nuclei (0), dna_damage (1) and microglia channel (2)
ki67_stack = img[0, :, ::slicing_factor, ::slicing_factor]
nuclei_stack = img[2, :, ::slicing_factor, ::slicing_factor]
caspase_stack = img[3, :, ::slicing_factor, ::slicing_factor]

# Perform maximum intensity projections
ki67_mip = np.max(ki67_stack, axis = 0)
nuclei_mip = np.max(nuclei_stack, axis = 0)
caspase_mip = np.max(caspase_stack, axis = 0)

# Free up memory by deleting the img variable
del img
# Call the garbage collector
gc.collect()

0

In [4]:
#TODO: Make 3D stack isotropic

# Voxel size from .czi file metadata
input_image = nuclei_stack

voxel_size_x = 0.21
voxel_size_y = 0.21
voxel_size_z = 2.17

# Adjust so voxel size_x and size_y so they are equal to 1 to avoid compression upon rescaling

multiplier = 0.5 / voxel_size_x

voxel_size_x = voxel_size_x * multiplier
voxel_size_y = voxel_size_y * multiplier
voxel_size_z = voxel_size_z * multiplier

print(voxel_size_x, voxel_size_y, voxel_size_z)

0.5 0.5 5.166666666666666


In [5]:
resampled = cle.scale(input_image, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, auto_size=True)
background_subtracted = cle.top_hat_box(resampled, radius_x=5, radius_y=5, radius_z=5)
segmented = cle.voronoi_otsu_labeling(background_subtracted, spot_sigma=3, outline_sigma=1)
resampled_ki67 = cle.scale(ki67_stack, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, auto_size=True)


MemoryError: clEnqueueNDRangeKernel failed: MEM_OBJECT_ALLOCATION_FAILURE

In [ ]:
nuclei_labels_3d = cle.pull(segmented)
resampled_ki67 = cle.pull(resampled_ki67)

# Extract regionprops
props = regionprops_table(label_image=nuclei_labels_3d, intensity_image=resampled_ki67, properties=["label", "intensity_mean", "intensity_max", "centroid", "area_filled"])

# Construct a dataframe
df = pd.DataFrame(props)

df

In [ ]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(nuclei_labels_3d)
viewer.add_image(resampled)
viewer.add_image(resampled_ki67)
